## 3.2. Preprocesamiento de datos

Para trabajar con los datos históricos de las acciones de **BBVA** y **Banco Santander**, se realizó un proceso exhaustivo de preprocesamiento con el fin de garantizar la calidad, consistencia y adecuación de los datos para su uso en modelos predictivos.

### • Conversión de fechas
La columna **Date**, que especifica la fecha de cada transacción, se transformó al formato de fecha estándar y se estableció como índice principal del conjunto de datos.  
Esta operación facilita la manipulación y el análisis temporal de las series históricas de precios.

### • Verificación de valores nulos
Se verificó la existencia de valores faltantes en las columnas clave (**Open**, **High**, **Low**, **Close**, **Adj Close**, **Volume**).  
Este paso fue crucial para evitar inconsistencias en el conjunto de datos.  
En los archivos `BBVA_core_clean.csv` y `SAN_core_clean.csv` no se encontraron valores nulos, lo que permitió continuar directamente con los cálculos posteriores.

### • Normalización de datos mediante *Min-Max Scaling*
Para garantizar que todas las variables estuvieran en una escala comparable, se aplicó el método de escalado **Min-Max** a las columnas de precios (**Open**, **High**, **Low**, **Close**, **Adj Close**) y al volumen (**Volume**).  
Este método es especialmente adecuado en contextos financieros, donde los datos presentan un rango acotado y la comparación entre variables requiere homogeneidad.

**Fórmula:**

$
X' = \frac{X - X_{min}}{X_{max} - X_{min}}
$

**Donde:**
- \( X' \): Valor escalado entre 0 y 1  
- \( X \): Valor original  
- \( X_{min} \): Valor mínimo de la columna  
- \( X_{max} \): Valor máximo de la columna  

Este proceso garantiza que todas las variables tengan el mismo rango, mejorando la estabilidad numérica y el rendimiento de los algoritmos de aprendizaje automático.  
Además, evita que las variables con rangos más amplios dominen sobre aquellas de menor magnitud.


In [1]:
# 0) Imports
import os
import json
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 1) Rutas y archivos
RAW_DIR = Path("../data/interim/precios_limpios")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

files = {
    "BBVA": RAW_DIR / "BBVA_core_clean.csv",
    "SAN" : RAW_DIR / "SAN_core_clean.csv",
}

files

{'BBVA': WindowsPath('../data/interim/precios_limpios/BBVA_core_clean.csv'),
 'SAN': WindowsPath('../data/interim/precios_limpios/SAN_core_clean.csv')}

In [3]:
# 2) Utilidades
KEY_COLS = ["Open","High","Low","Close","Adj Close","Volume"]
PRESERVE_COLS = ["Dividends","Stock Splits","Dividends_bin"]  # No se escalan
ALL_EXPECTED = ["Date","Adj Close","Close","Dividends","High","Low","Open","Stock Splits","Volume","Dividends_bin"]

def ensure_date_index(df: pd.DataFrame) -> pd.DataFrame:
    """Comprueba/corrige Date como datetime y lo pone como índice."""
    if "Date" not in df.columns:
        raise ValueError("No se encontró la columna 'Date' en el CSV.")
    if not pd.api.types.is_datetime64_any_dtype(df["Date"]):
        # intenta convertir
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce", utc=False, infer_datetime_format=True)
    # seguridad: elimina filas sin fecha válida
    df = df.dropna(subset=["Date"]).copy()
    df = df.sort_values("Date")
    df = df.set_index("Date")
    return df

def minmax_scale_df(df: pd.DataFrame, cols: list[str]) -> tuple[pd.DataFrame, dict]:
    """Aplica Min-Max a las columnas indicadas y devuelve df escalado + params."""
    scaler = MinMaxScaler()
    df_scaled = df.copy()
    scaler.fit(df_scaled[cols])
    df_scaled[cols] = scaler.transform(df_scaled[cols])
    # guardar parámetros para reproducibilidad
    params = {c: {"min": float(scaler.data_min_[i]), "max": float(scaler.data_max_[i])} 
              for i, c in enumerate(cols)}
    return df_scaled, params

def quick_null_report(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    """Conteo de nulos en columnas clave."""
    return df[cols].isna().sum()


In [4]:
# 3) Carga, validación de columnas y conversión de fechas
data_raw = {}
for name, path in files.items():
    df = pd.read_csv(path)
    
    # Validación de columnas esperadas (permite columnas extra)
    missing = [c for c in ALL_EXPECTED if c not in df.columns]
    if missing:
        raise ValueError(f"{name}: faltan columnas {missing}")
    
    # Asegurar Date como índice datetime
    df = ensure_date_index(df)

    data_raw[name] = df
    print(f"{name} -> filas: {len(df):,} | rango fechas: {df.index.min().date()} → {df.index.max().date()}")


BBVA -> filas: 6,634 | rango fechas: 2000-01-03 → 2025-10-30
SAN -> filas: 6,634 | rango fechas: 2000-01-03 → 2025-10-30


C:\Users\lopec\AppData\Local\Temp\ipykernel_21996\2821945429.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce", utc=False, infer_datetime_format=True)
C:\Users\lopec\AppData\Local\Temp\ipykernel_21996\2821945429.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce", utc=False, infer_datetime_format=True)


In [5]:
# 4) Verificación de valores nulos en columnas clave
for name, df in data_raw.items():
    nulls = quick_null_report(df, KEY_COLS)
    print(f"\nNulls en {name} (solo columnas clave):\n{nulls}")



Nulls en BBVA (solo columnas clave):
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

Nulls en SAN (solo columnas clave):
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [6]:
# 5) Min-Max Scaling en precios y volumen
scaled = {}
scaler_params = {}

for name, df in data_raw.items():
    # Copia de trabajo: nos quedamos con todas las columnas
    df_work = df.copy()

    # Escalar solo columnas KEY_COLS; el resto se preserva tal cual
    df_scaled, params = minmax_scale_df(df_work, KEY_COLS)
    scaled[name] = df_scaled
    scaler_params[name] = params

    # Guardado CSV escalado
    out_csv = OUT_DIR / f"{name}_core_scaled.csv"
    df_scaled.to_csv(out_csv, index=True)
    print(f"{name}: guardado escalado -> {out_csv}")

BBVA: guardado escalado -> ..\data\processed\BBVA_core_scaled.csv
SAN: guardado escalado -> ..\data\processed\SAN_core_scaled.csv


In [7]:
# 6) Guardar parámetros de escalado (min y max por columna) en JSON para reproducibilidad
params_path = OUT_DIR / "scalers_minmax_params.json"
with open(params_path, "w", encoding="utf-8") as f:
    json.dump(scaler_params, f, indent=2, ensure_ascii=False)
print("Parámetros de escalado guardados en:", params_path)

Parámetros de escalado guardados en: ..\data\processed\scalers_minmax_params.json


In [8]:
# 7) (Opcional) Comprobación rápida del resultado
for name, df in scaled.items():
    desc = df[KEY_COLS].describe().loc[["min","max"]]
    print(f"\n{name} - rangos tras Min-Max (esperado ~0..1):\n{desc}")


BBVA - rangos tras Min-Max (esperado ~0..1):
     Open  High  Low  Close  Adj Close  Volume
min   0.0   0.0  0.0    0.0        0.0     0.0
max   1.0   1.0  1.0    1.0        1.0     1.0

SAN - rangos tras Min-Max (esperado ~0..1):
     Open  High  Low  Close  Adj Close  Volume
min   0.0   0.0  0.0    0.0        0.0     0.0
max   1.0   1.0  1.0    1.0        1.0     1.0
